In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import time
from scipy.special import kv
from scipy.integrate import quad
from importlib import reload

from pulsars import Pulsars, c, e_charge, e_mass, h
import speiser_fun as sf 
import speiser_fun_cyl as sfc
import speiser_integ as si
import speiser_plots as sp
import speiser_model as sm
import speiser_spec as ss

In [2]:
k = 500.
name = 'crab'
pulsar = Pulsars(k)[name]

gamma1 = 1.
B_0 = pulsar['Blc']
omegaB = (e_charge*B_0/(e_mass*c))    #γυροσυχνότητα
Rlc = pulsar['rlc']*omegaB/c
Delta = pulsar['rlc']/k*omegaB/c
q = 1.

# gamma0 = np.array([1000., 1.])   #αρχικός παράγοντας Lorentz
# init_pos = np.array([0.5])

gamma0 = np.array([10., 1000., 10000., 1.])
init_pos = np.array([0.125, 0.5, 0.75, 1.])

delta = 50.    #πάχος του φύλλου ρεύματος, αδιάστατο, σε μονάδες [c/ωΒ]

T = 100
N = 10**5
t = np.linspace(0.0, T*Delta, N)

# with open('sim_info.txt', 'w') as nfo:
#     nfo.write([k, name, q, delta, T, N, gamma0, init_pos])

In [3]:
for w in range(len(init_pos)):
    
    print(init_pos[w])
    
    (r, ur, phi, uphi, z_cyl, uz_cyl), dic_cyl = si.oloklirosi(gamma0, Rlc, Delta, delta, B_0, t, init_pos[w], q, T, 
                                                               sm.speiser_cyl, coord = 'cyl')

    with open('cyl_{}_pos.npz'.format(int(100*init_pos[w])), 'wb') as f:
        np.savez(f, r = r, ur = ur, phi = phi, uphi = uphi, z_cyl = z_cyl, uz_cyl = uz_cyl)
    

0.125
runtime for gamma0 = 10 is 0h 8m 10s
runtime for gamma0 = 1000 is 0h 26m 25s
runtime for gamma0 = 10000 is 0h 12m 29s
total runtime = 0h 47m 5s
0.5
runtime for gamma0 = 10 is 0h 4m 54s
runtime for gamma0 = 1000 is 0h 24m 10s
runtime for gamma0 = 10000 is 0h 13m 7s
total runtime = 0h 42m 12s
0.75
runtime for gamma0 = 10 is 0h 4m 18s
runtime for gamma0 = 1000 is 0h 23m 55s
runtime for gamma0 = 10000 is 0h 13m 20s
total runtime = 0h 41m 34s
1.0
runtime for gamma0 = 10 is 0h 3m 38s
runtime for gamma0 = 1000 is 0h 23m 35s
runtime for gamma0 = 10000 is 0h 15m 6s
total runtime = 0h 42m 20s


In [4]:
#υπολογισμός ακτίνας καμπυλότητας, κρίσιμης συχνότητας και απωλειών

ss = reload(ss)
for w in range(len(init_pos)):
    
    if not (w == 7):
        a = np.load('cyl_{}_pos.npz'.format(int(100*init_pos[w])))
        r, ur, phi, uphi, z_cyl, uz_cyl = a['r'], a['ur'], a['phi'], a['uphi'], a['z_cyl'], a['uz_cyl']

        r_curv, nu_crit, p_rad = ss.rc_nc_pr(r, phi, z_cyl, ur, uphi, uz_cyl, Rlc, Delta, delta, gamma0, B_0)

        with open('rc_nc_pr_{}_pos.npz'.format(int(100*init_pos[w])), 'wb') as f2:
            np.savez(f2, r_curv = r_curv, nu_crit = nu_crit,  p_rad = p_rad)

        en, ph_num, ph_num_out, ph_en, ph_en_out, ph_num_tot, ph_en_tot = ss.spectrum(r, nu_crit, p_rad, gamma0, Rlc, t)

        with open('spectrum_{}_pos.npz'.format(int(100*init_pos[w])), 'wb') as f3:
            np.savez(f3, en = en, ph_num = ph_num, ph_num_out = ph_num_out, ph_en = ph_en, ph_en_out = ph_en_out,
                     ph_num_tot = ph_num_tot, ph_en_tot = ph_en_tot)